In [7]:
import pandas as pd
import sqlite3

users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [8]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [9]:

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()


with open("restaurants.sql", "r") as file:
    sql_script = file.read()


cursor.executescript(sql_script)


restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [10]:
orders.user_id = users.user_id
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)

orders_users.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,1.0,450,18-02-2023,842.97,New Foods Chinese,User_1,Chennai,Regular
1,2,2.0,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2,Pune,Gold
2,3,3.0,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_3,Bangalore,Gold
3,4,4.0,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_4,Bangalore,Regular
4,5,5.0,293,25-12-2023,1321.91,Royal Eatery South Indian,User_5,Pune,Gold


In [11]:
orders.restaurant_id = restaurants.restaurant_id
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,1.0,450,18-02-2023,842.97,New Foods Chinese,User_1,Chennai,Regular,Restaurant_450,Mexican,3.2
1,2,2.0,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2,Pune,Gold,Restaurant_309,Indian,4.5
2,3,3.0,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_3,Bangalore,Gold,Restaurant_107,Mexican,4.0
3,4,4.0,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_4,Bangalore,Regular,Restaurant_224,Chinese,4.8
4,5,5.0,293,25-12-2023,1321.91,Royal Eatery South Indian,User_5,Pune,Gold,Restaurant_293,Italian,3.0


In [12]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)
